# Packaging and Deployment

You have been tasked in federating the In this exercise we will be covering the basic components of the flower federated learning package. We will build a basic client, that can interface with our live server. We will also package the client.

We will complete the following steps:  

    1.  Write a basic client
    2.  Dockerize the client
    3.  Deploy and run federated learning rounds with (hopefully) all workshop participants!

## Writing a basic client

The client is flower's abstract class that executes the code on the edge. It runs code to train the local model, updates the model parameters from parameters received from the server and runs evaluations on local data. Because the client is an abstract class, some methods do not have a default implementation. These methods are the methods that define how the local model is trained and how parameter updates get sent from the edge to the server.



**def get_parameters(self, config) -> List:** defines how parameters are extracted from the edge model and how they are sent back to the server. Our server already exists, and expects parameters in a two-dimensional list format, with the format being \[coefficients (NDArrays), intercept (NDArrays)\]. The config parameter is mandatory in this method, but we will not use it in our implementation. Configuration parameters requested by the server. This can be used to tell the client which parameters are needed along with some Scalar attributes.

**def fit(self, parameters, config): -> List, Integer, Dict** defines how the model will be refit, using the parameters passed by the server. The parameters argument is passed by the global (server) model. Configuration parameters allow the server to influence training on the client. It can be used to communicate arbitrary values from the server to the client, for example, to set the number of (local) training epochs. It returns the following three outputs:   

&ensp; parameters (NDArrays) – The locally updated model parameters.  

&ensp; num_examples (int) – The number of examples used for training.  

&ensp; metrics (Dict[str, Scalar]) – A dictionary mapping arbitrary string keys to values of type bool, bytes, float, int, or str. It can be used to communicate arbitrary values back to the server. 
 

**def evaluate(self, parameters, config):** Evaluate the provided parameters using the locally held dataset. Returns these outputs:  

&ensp; loss (float) – The evaluation loss of the model on the local dataset.  

&ensp; num_examples (int) – The number of examples used for evaluation.  

&ensp; metrics (Dict[str, Scalar]) – A dictionary mapping arbitrary string keys to values of type bool, bytes, float, int, or str. It can be used to communicate arbitrary values back to the server.  


We have already written a client class with the skeleton methods below. Implement these methods, and when you're done run the cell. It will write the contents to a client.py file.

In [2]:
%%writefile client.py

from flwr.common import NDArrays, Scalar
from sklearn.linear_model import LinearRegression
import numpy as np
import flwr as fl
from sklearn.datasets import fetch_california_housing
from workshop.client.client import partition
from workshop.client.utils import set_initial_params

from typing import Dict, List, Tuple


class CaliforniaHousingClient(fl.client.NumPyClient):
    def __init__(self, partition_id: int):
        self.model = LinearRegression()
        set_initial_params(self.model)

        X, y = fetch_california_housing(return_X_y=True)

        partition_id = np.random.choice(10)

        self.X_train, self.y_train = X[:15000], y[:15000]
        self.X_test, self.y_test = X[15000:], y[15000:]

        self.X_train, self.y_train = partition(
            self.X_train, self.y_train, 10)[partition_id]
        
    def get_parameters(self, config) -> List:
        """Returns the paramters of a sklearn LinearRegression model."""
        return parameters

    def fit(self, parameters, config) -> tuple[NDArrays, int, dict]:
        """Refit the model locally with the central parameters and return them"""
        updated_parameters = parameters
        num_examples = 0
        metrics = {} # Won't be used in this example, we can return it empty

        return updated_parameters, num_examples, metrics
    
    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]) -> Tuple[float, int, Dict[str, Scalar]]:
        mse = 0
        num_examples = 0
        metrics = {}
        

        return mse, num_examples, metrics

Writing client.py


Now that we have a working client, we can package and deploy it! Like any good program, our client.py needs an entry point (if \_\_name\_\_ == "\_\_main\_\_") block. In it, we should construct our client, and connect it to the server! 

The client n

<span style= 'color:red'> WARNING </font>

The cell below will append the code written in it to the client every time you run it. If you made a mistake, make sure to remove the entry point block from client.py before running this cell again.

In [ ]:
%%writefile -a client.py

if __name__ == "__main__":
    while True:
        try:
            # pick up the server address from system arguments

            server_address

            client = CaliforniaHousingClient(partition_id=2)
            fl.client.start_numpy_client(
                server_address=server_address, client=client)
            break
        except Exception as e:
            logging.warning(
                "Could not connect to server: sleeping for 5 seconds...")
            time.sleep(5)

There is a dockerfile included in the exercise folder that you can use. As you can see, the dockerfile runs the client.py file. You can go ahead and copy-paste the **CaliforniaHousingClient** class you wrote into this file. The file also has an entry point (if \_\_name\_\_ == "\_\_main\_\_"). Under the entry point, we construct our client and connect it to the server. We therefore need to pass the server ip address. Let's pass this is a system argument to client.py in the dockerfile.

The dockerfile is already partially written. What is missing, is the environment v the 

In [1]:
%%writefile dockerfile

FROM python:3.9.16-slim

WORKDIR /usr/src/app

COPY . .

## ADD IP ENV variable

RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 8080

CMD ["python3", "client.py"] 

Writing dockerfile


Now let's build the docker image

In [7]:
!docker build -t client . 

[+] Building 0.0s (0/0)                                                         
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from dockerfile                       0.0s
 => => transferring dockerfile: 69B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9.16-slim      0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from dockerfile                       0.0s
 => => transferring dockerfile: 69B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B     

In [8]:
!docker run client

Traceback (most recent call last):
  File "/usr/src/app/client.py", line 7, in <module>
    from workshop.client.client import partition
ModuleNotFoundError: No module named 'workshop'
